# 1. pyloess 程序测试

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyloess

## 1.1. 1-D序列测试

In [ ]:
x = np.linspace(-10, 10, 1000)
y = np.sin(x) + np.random.random(1000) * 0.5

In [ ]:
f = plt.figure(figsize=(20,6))
plt.scatter(x, y, s=5)
plt.show()

### 1.1.1. Smooth_edge对比

In [ ]:
loess = pyloess.LocallyWeightedRegression(k=50)

y_pred1 = loess.PredictWithoutSmooth(x.reshape((-1, 1)), y, x.reshape((-1, 1)))
y_pred2 = loess.PredictWithinSmooth(x.reshape((-1, 1)), y, x.reshape((-1, 1)))

f = plt.figure(figsize=(20,6))
plt.scatter(x, y, s=5)
plt.plot(x, y_pred1, color='green',  label='not smooth edge')
plt.plot(x, y_pred2, color='orange', label='smooth edge')
plt.legend()
plt.show()

### 1.1.2. 部分预测

In [ ]:
x1 = np.linspace(5, 10.5, 50)
loess = pyloess.LocallyWeightedRegression(k=50)
y_pred1 = loess.PredictWithoutSmooth(x.reshape((-1, 1)), y, x1.reshape((-1, 1)))
y_pred2 = loess.PredictWithinSmooth(x.reshape((-1, 1)), y, x1.reshape((-1, 1)))

f = plt.figure(figsize=(20,6))
plt.scatter(x, y, s=5)
plt.plot(x1, y_pred1, color='green',  label='not smooth edge')
plt.plot(x1, y_pred2, color='orange', label='smooth edge')
plt.legend()
plt.show()

### 1.1.3 Robust测试

In [ ]:
y[100:105] = 1.5 + y[100:105]
loess = pyloess.LocallyWeightedRegression(k=50)

y_pred1 = loess.Fit(x.reshape((-1, 1)), y)
loess.robust = False
y_pred2 = loess.Fit(x.reshape((-1, 1)), y)

f = plt.figure(figsize=(20,6))
plt.scatter(x, y, s=5)
plt.plot(x, y_pred1, color='green',  label='robust')
plt.plot(x, y_pred2, color='orange', label='not robust')
plt.legend()
plt.show()

## 1.2. 2-D数据测试（待完成）

# 2. STL程序测试

## 2.1. statsmodels 中 STL 类

In [ ]:
import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams['figure.figsize'] = [20, 10]

from statsmodels.datasets import co2
# 加载数据
data = co2.load(True).data
data = data.resample('M').mean().ffill()
data = data.values.flatten()

In [ ]:
from statsmodels.tsa.seasonal import STL
res = STL(data, period=12).fit()
res.plot()

## 2.2. 自己 code pystl
### 2.2.1. 常规

In [ ]:
from pystl import SeasonalTrendDecomposition
stl = SeasonalTrendDecomposition(
    period_length = 12, 
    k1=6, k2=12, k3=12, 
    n_inner = 2, 
    n_outer = 3
)

stl.Fit(data)
results = stl.CombineResutls()
results.plot(subplots=True)

### 2.2.2 缺失值

In [58]:
import random
# 离散缺失
data_missing1 = data.copy()
data_missing2 = data.copy()

for i in range(data.shape[0]):
    if random.random() > 0.5:
        data_missing1[i] = np.nan
    if random.random() > 0.7:
        data_missing2[i] = np.nan
# 连续缺失
data_missing1[100:250] = np.nan
data_missing2[100:110] = np.nan

stl1 = SeasonalTrendDecomposition(
    period_length = 12, 
    k1=6, k2=12, k3=12, 
    n_inner = 2, 
    n_outer = 2
)
stl1.Fit(data_missing1)
results1 = stl1.CombineResutls()

stl2 = SeasonalTrendDecomposition(
    period_length = 12, 
    k1=6, k2=12, k3=12, 
    n_inner = 2, 
    n_outer = 2
)
stl2.Fit(data_missing2)
results2 = stl2.CombineResutls()

results1.plot(subplots=True)
results2.plot(subplots=True)

### 2.2.3. 包外样本分解（待完成）